### Домашнее задание №3

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
plt.style.use('seaborn-ticks')
plt.rcParams.update({'font.size': 14})

#### Функции

In [3]:
def standard_scaler(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

1. Измените функцию `calc_logloss` так, чтобы нули по возможности не попадали в `np.log` (как вариант - использовать `np.clip` или `np.where`).  

In [5]:
# Задаем значение замены, в случае, когда np.log == 0:
replacement_number = 1e-10

In [6]:
def calc_logloss(y, y_pred):
    err = np.mean(- y * np.where(y_pred == 0, np.log(replacement_number), np.log(y_pred)) - \
                  (1.0 - y) * np.where((1.0 - y_pred) == 0, np.log(replacement_number), np.log(1.0 - y_pred)))
    return err

* Проверка работы функции и ее критических значений

In [7]:
# Пример применения
y_true = 1
y_pred = 0.9
calc_logloss(y_true, y_pred)

0.10536051565782628

In [8]:
# Пример применения
y_true = 1
y_pred = 0
calc_logloss(y_true, y_pred)

<ipython-input-6-357fadac03f3>:2: RuntimeWarning: divide by zero encountered in log
  err = np.mean(- y * np.where(y_pred == 0, np.log(replacement_number), np.log(y_pred)) - \


23.025850929940457

In [9]:
# Пример применения
y_true = 0
y_pred = 1
calc_logloss(y_true, y_pred)

<ipython-input-6-357fadac03f3>:3: RuntimeWarning: divide by zero encountered in log
  (1.0 - y) * np.where((1.0 - y_pred) == 0, np.log(replacement_number), np.log(1.0 - y_pred)))


23.025850929940457

2. Подберите аргументы функции `eval_LR_model` для логистической регрессии таким образом, чтобы log loss был минимальным. Покажите влияние гиперпараметров на ошибку алгоритма (оптимально здесь использовать критерий остановки обучения).

In [10]:
def eval_LR_model_minlogloss(X, y, iterations, alpha=1e-4, diff = 1e-6):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    stop = np.inf
    error = []    
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        error.append(err)
        w -= alpha * (1/n * np.dot(X.T, (y_pred - y)))
        if i % (iterations / 10) == 0:
            print(i, w, err)
        if abs(stop - err) <= diff:
            print(i, w, err, f" Остановка на {i} итерации")
            break
        stop = err       
    return w, error, i

In [11]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика
X

array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
       [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
       [1.00e+00, 1.00e+00, 6.00e+02, 1.00e+00],
       [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
       [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
       [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
       [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
       [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]])

In [12]:
X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])
X_st[:, 2]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

In [13]:
best_result = {"Веса": np.inf, "Итерация": np.inf, "Ошибка": np.inf, "Скорость":np.inf, "Остановка":np.inf}

In [14]:
for iterations in range(1000, 10000, 1000):
    for alpha in np.logspace(start=-3, stop=-1, num=5):
        w, errs, early_stop = eval_LR_model_minlogloss(X_st, y, iterations, alpha)
        if errs[-1] < best_result.get("Ошибка"):
            best_result = {"Веса": w, "Итерация": iterations, "Ошибка": errs[-1], "Скорость": alpha, "Остановка":early_stop}

print("\nЛучшие параметры:")
best_result

100 [ 0.4588585  -0.2544203   0.6453169   1.46694374] 1.0410271098401283
200 [ 0.42366472 -0.36366056  0.64254667  1.41503365] 0.8819616824356442
300 [ 0.39197407 -0.45912506  0.6408019   1.36968388] 0.7595098840306068
400 [ 0.36451116 -0.53324936  0.64182901  1.3332885 ] 0.6828881509068779
500 [ 0.34108547 -0.58545911  0.64603577  1.3058302 ] 0.6418288972649147
600 [ 0.32083524 -0.62105749  0.65259781  1.28534741] 0.6201376264698928
700 [ 0.30292451 -0.64542451  0.66061271  1.26984863] 0.6078121821371583
800 [ 0.28674785 -0.66226082  0.66947011  1.25790601] 0.6000844770958682
900 [ 0.27188803 -0.67393645  0.67879458  1.24855638] 0.5947360850543497
1000 [ 0.25805314 -0.68199349  0.68835345  1.24114037] 0.5906912249945011
100 [ 0.38712876 -0.4730563   0.64073334  1.36296779] 0.7456413171525382
200 [ 0.31468074 -0.63041915  0.65505228  1.27963714] 0.6155376081804886
300 [ 0.26494822 -0.67849772  0.68343943  1.244594  ] 0.5926882391212136
400 [ 0.2248844  -0.69281881  0.71387645  1.227660

1800 [-0.10795739 -0.69013564  0.97357614  1.26106112] 0.5383097754313371
2100 [-0.15542322 -0.69655427  1.00059184  1.28751825] 0.5343766746036489
2400 [-0.19888742 -0.70425966  1.02237164  1.3165928 ] 0.5309273816342566
2700 [-0.2393708  -0.71276824  1.04011502  1.34736982] 0.5277906814980151
3000 [-0.27762206 -0.72173821  1.0547053   1.37918352] 0.524870780379483
300 [ 0.06257746 -0.68605867  0.85181735  1.21283095] 0.5560473747533587
600 [-0.1239972  -0.69202069  0.98318257  1.26924759] 0.536984186497779
900 [-0.25830834 -0.71709934  1.04769231  1.36276535] 0.5263621224044184
1200 [-0.37188064 -0.7461597   1.08260067  1.46531853] 0.5178498846713239
1500 [-0.47657171 -0.77421839  1.10230835  1.566837  ] 0.5103621732323236
1800 [-0.57684595 -0.79979671  1.11275552  1.66410902] 0.503597783659264
2100 [-0.67442829 -0.82262769  1.11668647  1.75639311] 0.4974027905970713
2400 [-0.77001652 -0.8428484   1.11557702  1.84379918] 0.4916712819232016
2700 [-0.86391541 -0.86071239  1.11035857  1

500 [-0.50412    -0.78149848  1.1061256   1.59379469] 0.5085082074095577
1000 [-1.00570283 -0.88433651  1.09591604  2.04680949] 0.4787265990513074
1500 [-1.46591581 -0.93822807  1.00564105  2.40179928] 0.4565996357207549
2000 [-1.89211153 -0.96714688  0.88204274  2.69883962] 0.43848762403073616
2500 [-2.28801306 -0.9833808   0.74659476  2.9594008 ] 0.4230908227834881
3000 [-2.65667555 -0.99333042  0.61005408  3.19528883] 0.40978175247484294
3500 [-3.00081104 -1.00039027  0.47771973  3.41333049] 0.39816746119967517
4000 [-3.32288601 -1.00634551  0.35198478  3.61765028] 0.3879591701752688
4500 [-3.62514809 -1.01210313  0.2336855   3.81085745] 0.3789284996465496
5000 [-3.9096333  -1.01809176  0.1228438   3.99469141] 0.3708897077794019
500 [-1.5375957  -0.94440497  0.98735478  2.45373702] 0.4535011239613406
1000 [-2.77136565 -0.99588853  0.56641089  3.26813012] 0.40587815270727734
1500 [-3.76622998 -1.01499348  0.17863749  3.90176516] 0.3749222940537399
2000 [-4.59094777 -1.0359998  -0.138

2100 [-4.74008164 -1.0407042  -0.19438587  4.54566439] 0.3497829571572577
2800 [-5.67269606 -1.07679341 -0.53652756  5.19238754] 0.32962853936998515
3500 [-6.46048974 -1.11561113 -0.81458335  5.76040867] 0.3149982951070749
4200 [-7.15035381 -1.15493275 -1.05121316  6.27189678] 0.30362540456018605
4900 [-7.77085007 -1.19385327 -1.25953029  6.74138585] 0.294326561635074
5600 [-8.34016004 -1.23203577 -1.44751986  7.17868525] 0.2864338954481406
6300 [-8.87033866 -1.26937584 -1.62030227  7.59059063] 0.27954497519520655
7000 [-9.36966717 -1.30586395 -1.78130777  7.98193089] 0.2734038243094169
800 [ 0.28674785 -0.66226082  0.66947011  1.25790601] 0.6000844770958682
1600 [ 0.1879499  -0.6953201   0.74498575  1.21841972] 0.575624870144612
2400 [ 0.11204861 -0.68987785  0.81056775  1.21120728] 0.5628776455640998
3200 [ 0.04751324 -0.68530277  0.86380541  1.21448096] 0.5540558173055711
4000 [-0.00868429 -0.68420414  0.90686656  1.22453943] 0.5476418419000334
4800 [-0.05839486 -0.68589164  0.94188

{'Веса': array([-10.67239108,  -1.40571631,  -2.1950503 ,   9.01453862]),
 'Итерация': 9000,
 'Ошибка': 0.2586628760279487,
 'Скорость': 0.1,
 'Остановка': 9000}

3. Создайте функцию `calc_pred_proba`, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`, на выходе - массив `y_pred_proba`).

In [15]:
border = 0.5

In [16]:
def calc_pred_proba(X, w):    
    y_pred_proba = sigmoid(np.dot(X_st, w))
    return y_pred_proba >= border

In [17]:
sigmoid(np.dot(X_st, w))

array([0.28632092, 0.13959138, 0.99618044, 0.20326622, 0.72894738,
       0.29615909, 0.99905752, 0.08899659, 0.33470563, 0.98825   ])

In [18]:
calc_pred_proba(X_st, w)

array([False, False,  True, False,  True, False,  True, False, False,
        True])

In [19]:
border = 0.8

In [20]:
calc_pred_proba(X_st, w)

array([False, False,  True, False, False, False,  True, False, False,
        True])

4. Создайте функцию `calc_pred`, возвращающую предсказанный класс (на вход подаются значения признаков `Х` и веса, которые уже посчитаны функцией `eval_LR_model`, на выходе - массив `y_pred`).

In [21]:
def calc_pred(X, w):
  y_pred = sigmoid(np.dot(X_st, w))    
  return [1 if elem > 0.55 else 0 for elem in y_pred]

In [22]:
y_pred = calc_pred(X, w)
y_pred

[0, 0, 1, 0, 1, 0, 1, 0, 0, 1]

5. Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.

In [23]:
print(f'y:      {y}\ny_pred: {np.array(y_pred)}')

y:      [0 0 1 0 1 0 1 0 1 1]
y_pred: [0 0 1 0 1 0 1 0 0 1]


In [24]:
import sklearn.metrics
from sklearn.metrics import precision_recall_curve

In [25]:
accuracy = sklearn.metrics.accuracy_score(y, y_pred)
accuracy

0.9

In [26]:
conf_matrix = sklearn.metrics.confusion_matrix(y, y_pred)
conf_matrix

array([[5, 0],
       [1, 4]], dtype=int64)

In [27]:
precision = conf_matrix[1,1] / (conf_matrix[1,1] + conf_matrix[0,1])
precision

1.0

In [28]:
recall = conf_matrix[1,1] / (conf_matrix[1,1] + conf_matrix[1,0])
recall

0.8

In [29]:
f1_score=sklearn.metrics.f1_score(y, y_pred)
f1_score

0.888888888888889

6. Могла ли модель переобучиться? Почему?

In [30]:
# Да могла, из-за того, что в датафрейме очень маленькое количествонаблюдений - всего 10. 